# Using optimizers

In [1]:
# Setting seeds to try and ensure we have the same results - this is not guaranteed across PyTorch releases.
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import numpy as np
np.random.seed(0)

In [2]:
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch import nn

mean, std = (0.5,), (0.5,)

# Create a transform and normalise data
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean, std)
                              ])

# Download FMNIST training dataset and load training data
trainset = datasets.FashionMNIST('~/.pytorch/FMNIST/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

# Download FMNIST test dataset and load test data
testset = datasets.FashionMNIST('~/.pytorch/FMNIST/', download=True, train=False, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

100%|██████████| 26421880/26421880 [00:01<00:00, 14073948.76it/s]


Extracting /root/.pytorch/FMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/FMNIST/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 281857.43it/s]


Extracting /root/.pytorch/FMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/FMNIST/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 4989057.67it/s]


Extracting /root/.pytorch/FMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/FMNIST/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 19244453.65it/s]


Extracting /root/.pytorch/FMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/FMNIST/FashionMNIST/raw



In [3]:
class FMNIST(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(784, 128)
    self.fc2 = nn.Linear(128,64)
    self.fc3 = nn.Linear(64,10)

  def forward(self, x):
    x = x.view(x.shape[0], -1)

    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    x = F.log_softmax(x, dim=1)

    return x

#model = FMNIST()

In [4]:
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.LogSoftmax(dim=1))

In [5]:
images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)

In [6]:
from torch import optim

criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01) # here lr is the learning rate


In [7]:
# forward pass, calculate loss, backword pass
output = model(images)
loss = criterion(output, labels)
loss.backward()
print('Initial weights : ',model[0].weight)
print('Initial weights gradient : ',model[0].weight.grad)


Initial weights :  Parameter containing:
tensor([[-0.0003,  0.0192, -0.0294,  ...,  0.0219,  0.0037,  0.0021],
        [-0.0198, -0.0150, -0.0104,  ..., -0.0203, -0.0060, -0.0299],
        [-0.0201,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [ 0.0018, -0.0295,  0.0085,  ..., -0.0037,  0.0036,  0.0300],
        [-0.0233, -0.0220, -0.0064,  ...,  0.0115, -0.0324, -0.0158],
        [ 0.0309,  0.0066,  0.0125,  ...,  0.0286,  0.0350, -0.0105]],
       requires_grad=True)
Initial weights gradient :  tensor([[-0.0030, -0.0030, -0.0030,  ..., -0.0030, -0.0030, -0.0030],
        [ 0.0022,  0.0022,  0.0022,  ...,  0.0024,  0.0022,  0.0022],
        [ 0.0002,  0.0002,  0.0002,  ...,  0.0002,  0.0002,  0.0002],
        ...,
        [ 0.0014,  0.0014,  0.0014,  ...,  0.0014,  0.0014,  0.0014],
        [ 0.0021,  0.0021,  0.0021,  ...,  0.0022,  0.0021,  0.0021],
        [ 0.0038,  0.0038,  0.0038,  ...,  0.0038,  0.0038,  0.0038]])


In [8]:
# Update the weights using step function of the optimizer
optimizer.step()

In [9]:
# after running the optimizer the weights are updated
print('Initial weights : ',model[0].weight)
print('Initial weights gradient : ',model[0].weight.grad)

Initial weights :  Parameter containing:
tensor([[-0.0002,  0.0192, -0.0294,  ...,  0.0220,  0.0038,  0.0021],
        [-0.0198, -0.0150, -0.0105,  ..., -0.0203, -0.0060, -0.0300],
        [-0.0202,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [ 0.0018, -0.0296,  0.0085,  ..., -0.0037,  0.0036,  0.0300],
        [-0.0233, -0.0221, -0.0064,  ...,  0.0115, -0.0324, -0.0158],
        [ 0.0309,  0.0065,  0.0125,  ...,  0.0285,  0.0349, -0.0106]],
       requires_grad=True)
Initial weights gradient :  tensor([[-0.0030, -0.0030, -0.0030,  ..., -0.0030, -0.0030, -0.0030],
        [ 0.0022,  0.0022,  0.0022,  ...,  0.0024,  0.0022,  0.0022],
        [ 0.0002,  0.0002,  0.0002,  ...,  0.0002,  0.0002,  0.0002],
        ...,
        [ 0.0014,  0.0014,  0.0014,  ...,  0.0014,  0.0014,  0.0014],
        [ 0.0021,  0.0021,  0.0021,  ...,  0.0022,  0.0021,  0.0021],
        [ 0.0038,  0.0038,  0.0038,  ...,  0.0038,  0.0038,  0.0038]])


In [10]:
# we updated the weights and we dont want them to accumulated so zero them up
optimizer.zero_grad()

In [11]:
print('Initial weights : ',model[0].weight)
print('Initial weights gradient : ',model[0].weight.grad)

Initial weights :  Parameter containing:
tensor([[-0.0002,  0.0192, -0.0294,  ...,  0.0220,  0.0038,  0.0021],
        [-0.0198, -0.0150, -0.0105,  ..., -0.0203, -0.0060, -0.0300],
        [-0.0202,  0.0149, -0.0333,  ..., -0.0203,  0.0012,  0.0080],
        ...,
        [ 0.0018, -0.0296,  0.0085,  ..., -0.0037,  0.0036,  0.0300],
        [-0.0233, -0.0221, -0.0064,  ...,  0.0115, -0.0324, -0.0158],
        [ 0.0309,  0.0065,  0.0125,  ...,  0.0285,  0.0349, -0.0106]],
       requires_grad=True)
Initial weights gradient :  None


In [12]:
model = FMNIST()
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 3

for i in range(num_epochs):
    cum_loss = 0
    batch_num = 0
    for batch_num, (images, labels) in enumerate(trainloader, 1):
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        cum_loss += loss.item()
        print(f'Batch : {batch_num}, Loss : {loss.item()}')

    print(f"Training loss: {cum_loss/len(trainloader)}")

Batch : 1, Loss : 2.2870395183563232
Batch : 2, Loss : 2.304734468460083
Batch : 3, Loss : 2.28812313079834
Batch : 4, Loss : 2.2898640632629395
Batch : 5, Loss : 2.2802863121032715
Batch : 6, Loss : 2.2768282890319824
Batch : 7, Loss : 2.2971038818359375
Batch : 8, Loss : 2.272695779800415
Batch : 9, Loss : 2.266519546508789
Batch : 10, Loss : 2.276360273361206
Batch : 11, Loss : 2.285900115966797
Batch : 12, Loss : 2.2687723636627197
Batch : 13, Loss : 2.2825448513031006
Batch : 14, Loss : 2.269296407699585
Batch : 15, Loss : 2.251699447631836
Batch : 16, Loss : 2.2710673809051514
Batch : 17, Loss : 2.2565464973449707
Batch : 18, Loss : 2.247075080871582
Batch : 19, Loss : 2.242631673812866
Batch : 20, Loss : 2.2493398189544678
Batch : 21, Loss : 2.2490806579589844
Batch : 22, Loss : 2.239584445953369
Batch : 23, Loss : 2.2394418716430664
Batch : 24, Loss : 2.252643585205078
Batch : 25, Loss : 2.2172629833221436
Batch : 26, Loss : 2.236956834793091
Batch : 27, Loss : 2.22307038307189